In [1]:
my_api_key = '598be18ba2417a981d0ada77f9f32b83'

In [2]:
# amap.com 行政区域查询
# http://lbs.amap.com/api/webservice/guide/api/district/
from pprint import pprint
import requests #用於API HTTP requests
import json     #用於输入出json

# 实践 amap行政区域查询API 中的 服务示例
# http://restapi.amap.com/v3/config/district?keywords=北京&subdistrict=2&key=<用户的key>

url_api = "http://restapi.amap.com/v3/config/district"
parameters = {'keywords': '中国', 
              'subdistrict': 1,
              'key': my_api_key}
r = requests.get (url_api, params=parameters)
data = r.json()


In [3]:
#  dict comprehension code --> int
print( { str(x['adcode']):x['name'] for x in data['districts'][0]['districts'] } )

{'110000': '北京市', '500000': '重庆市', '820000': '澳门特别行政区', '440000': '广东省', '350000': '福建省', '460000': '海南省', '140000': '山西省', '710000': '台湾省', '520000': '贵州省', '450000': '广西壮族自治区', '620000': '甘肃省', '410000': '河南省', '130000': '河北省', '340000': '安徽省', '430000': '湖南省', '310000': '上海市', '420000': '湖北省', '230000': '黑龙江省', '150000': '内蒙古自治区', '640000': '宁夏回族自治区', '360000': '江西省', '320000': '江苏省', '220000': '吉林省', '210000': '辽宁省', '370000': '山东省', '650000': '新疆维吾尔自治区', '120000': '天津市', '630000': '青海省', '610000': '陕西省', '540000': '西藏自治区', '510000': '四川省', '810000': '香港特别行政区', '530000': '云南省', '330000': '浙江省'}


In [5]:
places = { int(x['adcode']):x['name'] for x in data['districts'][0]['districts'] }
keywords = ['大学','中学']

# amap.com 搜索 api/search：包成函数
函数输入：
- 关键词
- 地区代码
- API key

函数输出/返回：
- dataframe
- amap_api_search_by_keyword_city_code(kw=关键词, cc=地区代码, api_key=API key)
- amap_api_search_by_keyword_city_code(kw='植物园', cc='440000', api_key='你自己申请到的key')


In [6]:
import requests #用於API HTTP requests
import pandas as pd


def amap_api_search_by_keyword_city_code(kw, cc, api_key):

    parameters = {'key': api_key}  # 高德地图API 获取Key
    parameters.update( {'keywords': kw,
                        'city':     cc,
                        'citylimit': True
                       } )

    pois = []   # 建构最终结果列表pois
    pg_no = 1   # 建构pg_no，每次给不一样的可选参数page值

    while True: # 不断迭代，直到 break跳出
        parameters.update({'page':pg_no})  #建构可选参数page值 为 pg_no

        r = requests.get ("http://restapi.amap.com/v3/place/text", params=parameters)
        data = r.json()
        #print(data)

        no_pois = int(data['count'])      # 计算 应有的实质数量

        if no_pois>0:
            pois.extend(data.get('pois', []))         # 使用Python列表之extend方法，把返回数据中的结果data['pois']存入pois
            no_pois_this_search = len(pois)   # 计算 累积实质结果数
            if (no_pois_this_search >= no_pois):  # 若 累积实质结果数 >= 应有的实质数量
                break                                   # 则结束迭代跳出
            else:
                pg_no +=1                               # 否则找下一页数据，把 pg_no增1

        else:
            break                                    # empty

    if len(pois)>0:
        # 使用pandas 模块处理数据
        df_input = pd.DataFrame(pois)

        # 选择部份栏位准备输出
        select_fields = ['location', 'address', 'adname', 'cityname', 'id', 'importance', 'name', 'pname', 'tel', 'type', 'typecode']
        df = df_input[select_fields]

        # 经纬度 增加新栏位字段 places 及 keywords
        df = df.assign( 经 = [x.split(',')[0] for x in df['location']])
        df = df.assign( 纬 = [x.split(',')[1] for x in df['location']])
        
        df = df.assign( places_code = [cc]*len(df['location']))
        df = df.assign( places = [places[cc]]*len(df['location']))
        df = df.assign( keywords = [kw]*len(df['location']))
        
    else:
        df = pd.DataFrame([])
    
    # 使用 return返回
    return(df)


## 使用函数

In [7]:
# 少量测试
{ c:places[c] for c in places.keys() if c in [810000, 820000]}

{810000: '香港特别行政区', 820000: '澳门特别行政区'}

In [10]:
import os.path
fn_output = "some_data.tsv"

for k in [keywords[0]]:  # keywords
    for p in { c:places[c] for c in places.keys() if c in [810000, 820000]}: # places
        d = amap_api_search_by_keyword_city_code(k, p, my_api_key)
        print ("读取 {地点} 中的 {关键字} 搜索结果...共{几个}个".format(地点=places[p], 关键字=k, 几个=len(d)))
        
        if os.path.exists(fn_output): 
            header_needed = False
        else:
            header_needed = True
        
        d.to_csv(fn_output, encoding='utf8', sep='\t', mode='a',
                                index = False,  index_label=False,
                                header = header_needed)

KeyboardInterrupt: 

# 放入储存文件夹名称


In [11]:
# 全部执行
import os.path
fn_output = "university.tsv"

for k in keywords:
    for p in places:
        d = amap_api_search_by_keyword_city_code(k, p, my_api_key)
        print ("读取 {地点} 中的 {关键字} 搜索结果...共{几个}个".format(地点=places[p], 关键字=k, 几个=len(d)))

        if os.path.exists(fn_output): 
            header_needed = False
        else:
            header_needed = True
        
        # header  若数据档不存在则输出时包括栏位标题，若已存在则不包括
        d.to_csv(fn_output, encoding='utf8', sep='\t', mode='a',
                                index = False,  index_label=False,
                                header = header_needed)
        

ConnectionError: HTTPConnectionPool(host='restapi.amap.com', port=80): Max retries exceeded with url: /v3/place/text?key=598be18ba2417a981d0ada77f9f32b83&keywords=%E5%A4%A7%E5%AD%A6&city=110000&citylimit=True&page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001936C3066A0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [ ]:
# 读取 所有数据

df_all = pd.read_csv(fn_output, encoding='utf8', sep="\t")

In [ ]:
# 数据长度
len(df_all)

In [24]:
df_all.columns

Index(['location', 'address', 'adname', 'cityname', 'id', 'importance', 'name',
       'pname', 'tel', 'type', 'typecode', '经', '纬', 'places_code', 'places',
       'keywords'],
      dtype='object')

In [25]:
# pivot_table 
df_all.pivot_table(values='name', index='places', columns='keywords', 
                         aggfunc=lambda x: len(x.unique()))

keywords,iphone售后,oppo售后,vivo售后,小米售后,魅族售后
places,,,,,
上海市,224.0,27.0,18.0,240.0,10.0
云南省,34.0,39.0,20.0,184.0,7.0
内蒙古自治区,29.0,49.0,25.0,134.0,5.0
北京市,76.0,17.0,10.0,216.0,1.0
台湾省,NaN,NaN,NaN,26.0,NaN
吉林省,33.0,38.0,18.0,97.0,5.0
四川省,77.0,45.0,28.0,339.0,12.0
天津市,14.0,18.0,9.0,64.0,2.0
宁夏回族自治区,8.0,6.0,7.0,29.0,NaN


In [26]:
# pivot_table 
df_all.pivot_table(values='name', index='type', columns='keywords', 
                         aggfunc=lambda x: len(x.unique()))

keywords,iphone售后,oppo售后,vivo售后,小米售后,魅族售后
type,,,,,
交通设施服务;交通服务相关;交通服务相关,NaN,NaN,NaN,1.0,NaN
交通设施服务;停车场;专用停车场,NaN,NaN,NaN,1.0,NaN
交通设施服务;停车场;停车场出入口,NaN,NaN,NaN,1.0,NaN
交通设施服务;停车场;停车场相关,NaN,NaN,NaN,1.0,NaN
交通设施服务;公交车站;公交车站相关,NaN,NaN,NaN,5.0,NaN
住宿服务;住宿服务相关;住宿服务相关,NaN,NaN,NaN,37.0,NaN
住宿服务;宾馆酒店;宾馆酒店,NaN,NaN,NaN,24.0,NaN
住宿服务;宾馆酒店;经济型连锁酒店,NaN,NaN,NaN,5.0,NaN
住宿服务;旅馆招待所;旅馆招待所,NaN,NaN,NaN,17.0,NaN


In [27]:
# 简化
[x.split(";")[0] if ";" in x else None for x in df_all['type']]

['生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '购物服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '科教文化服务',
 '生活服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '生活服务',
 '住宿服务',
 '餐饮服务',
 '购物服务',
 '科教文化服务',
 '购物服务',
 '购物服务',
 '餐饮服务',
 '购物服务',
 '购物服务',
 '公司企业',
 '购物服务',
 '公司企业',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '生活服务',
 '生活服务',
 '购物服务',
 '餐饮服务',
 '生活服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '商务住宅',
 '购物服务',
 '购物服务',
 '生活服务',
 '购物服务',
 '生活服务',
 '生活服务',
 '购物服务',
 '购物服务',
 '生活服务',
 '生活服务',
 '生活服务',
 '餐饮服务',
 '公司企业',
 '公司企业',
 '餐饮服务',
 '餐饮服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '金融保险服务',
 '购物服务',
 '购物服务',
 '科教文化服务',
 '生活服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '购物服务',
 '汽车维修',
 '购物服务',
 '

In [28]:
# pivot_table 简化type
df_all = df_all.assign( type_ = [x.split(";")[0] if ";" in x else None for x in df_all['type']])

df_all.pivot_table(values='name', index='type_', columns='keywords', 
                         aggfunc=lambda x: len(x.unique()))

keywords,iphone售后,oppo售后,vivo售后,小米售后,魅族售后
type_,,,,,
交通设施服务,NaN,NaN,NaN,9.0,NaN
住宿服务,NaN,NaN,NaN,85.0,NaN
体育休闲服务,NaN,NaN,NaN,33.0,NaN
公司企业,24.0,NaN,2.0,167.0,1.0
医疗保健服务,1.0,NaN,NaN,13.0,NaN
商务住宅,4.0,NaN,NaN,28.0,NaN
地名地址信息,NaN,1.0,NaN,128.0,NaN
摩托车服务,NaN,NaN,NaN,4.0,NaN
政府机构及社会团体,1.0,1.0,NaN,13.0,NaN


In [19]:
# 由上数据观之，动物园 和 植物园 多以 风景名胜 为主要分类，而 动物园 涉及的类型较其他为杂，有可能要排除一些不相关数据

